In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind, ttest_rel, mannwhitneyu, norm, normaltest

from analysis.analysis_utils import add_tt_mals_runtime_cols, line_plot_padding_tile_size_tt_mals_runtime_per_matrix, get_percentage_change_per_category, get_percentage_change_per_double_category, line_plot_tile_size_rank_percentage_per_matrix, normality_check_histogram


In [31]:
# df1 = pd.read_csv('../../data/sweep_0_10_data1.csv')  # 0-10, small matrices
df2 = pd.read_csv('../../data/sweep_0_10_data2.csv')  # 0-10, big matrices
# df3 = pd.read_csv('../../data/sweep_0_10_data3.csv')  # powers of 2, all matrices
# df4 = pd.read_csv('../../data/sweep_0_10_data4.csv')  # around powers of 2, small matrices
# df5 = pd.read_csv('../../data/sweep_0_10_data5.csv')  # 10 fractional matrix sizes, small matrices
# df5 = df5[df5['partial_gauss']<1.0]  # don't consider full row reductions
# df5 has fractional partial_gauss params - convert it to absolute terms
# df5["partial_gauss"] = (df5["partial_gauss"]* (df5["n"] / (1 - df5["partial_gauss"]))).astype("int")

# df = pd.concat([df1, df2, df5], axis=0)  # df3, df4
df = pd.read_csv('../../data/sweep_inc_data1.csv')
df = df[df['sparsity_ratio'] > 0.8]  # drop rows that are incomplete due to too low sparsity
df.drop(columns=['partial_gauss'], inplace=True)
df.rename(columns={"num_reduced_variables": "partial_gauss"}, inplace=True)
# concat 0-10 sweeps for larger matrices
df = pd.concat([df, df2], axis=0)  # df3, df4
df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp", "gauss_threshold", "min_sparsity", "partial_gauss_increments"], inplace=True)
df.shape

(2331080, 12)

In [32]:
# df1 = pd.read_csv('../../data/sweep_0_10_data1.csv')
# df2 = pd.read_csv('../../data/sweep_0_10_data2.csv')
# df = pd.concat([df1, df2], axis=0)
# df.drop(columns=["run_id", "run_name", "_runtime", "_step", "_timestamp", "gauss_threshold"], inplace=True)
# df.shape

In [33]:
df.tail()

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n
9403,True,True,2,Pres_Poisson,623417.0,5.0,1625.0,73.0,718845.0,73.0,NaN,14819.0
9404,True,True,2,Pres_Poisson,623417.0,5.0,325.0,203.0,718845.0,203.0,NaN,14819.0
9405,True,True,2,Pres_Poisson,623417.0,5.0,85.0,511.0,718845.0,511.0,NaN,14819.0
9406,True,True,2,Pres_Poisson,623417.0,5.0,19.0,2117.0,718845.0,2117.0,NaN,14819.0
9407,True,True,2,Pres_Poisson,623417.0,5.0,1.0,14819.0,718845.0,14819.0,NaN,14819.0


In [34]:
df = add_tt_mals_runtime_cols(df)

df["reduced_variable_percentage"] = df["partial_gauss"] / (df["n"]+ df["partial_gauss"])

In [35]:
# important: we only care about best tile size choice rows that get enabled by these approaches (unless we want to plot the complete picture)
# so drop the rest of the rows for this analysis
# idx = df.groupby(by=['amd', 'rcm', "padding", "matrix_name", "partial_gauss"])['log_obj_func'].idxmin()
# df = df.loc[idx].reset_index(drop=True)

In [36]:
df

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n,log_obj_func,obj_func,reduced_variable_percentage
0,False,False,10,bcsstk13,70264.0,0.0,18672.0,3.0,70264.0,61.0,0.982487,2013.0,27.933391,1.353061e+12,0.000000
1,False,False,10,bcsstk13,70264.0,0.0,3355.0,11.0,70264.0,61.0,0.982487,2013.0,25.268320,9.416556e+10,0.000000
2,False,False,10,bcsstk13,70264.0,0.0,755.0,33.0,70264.0,61.0,0.982487,2013.0,24.708777,5.381281e+10,0.000000
3,False,False,10,bcsstk13,70264.0,0.0,339.0,61.0,70264.0,61.0,0.982487,2013.0,24.674989,5.202494e+10,0.000000
4,False,False,10,bcsstk13,70264.0,0.0,75.0,183.0,70264.0,183.0,0.982487,2013.0,31.256934,3.755900e+13,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9403,True,True,2,Pres_Poisson,623417.0,5.0,1625.0,73.0,718845.0,73.0,NaN,14819.0,25.835484,1.660383e+11,0.000337
9404,True,True,2,Pres_Poisson,623417.0,5.0,325.0,203.0,718845.0,203.0,NaN,14819.0,31.879337,6.998744e+13,0.000337
9405,True,True,2,Pres_Poisson,623417.0,5.0,85.0,511.0,718845.0,511.0,NaN,14819.0,37.418218,1.780433e+16,0.000337
9406,True,True,2,Pres_Poisson,623417.0,5.0,19.0,2117.0,718845.0,2117.0,NaN,14819.0,45.946532,9.001713e+19,0.000337


In [37]:
# no module is applied - baseline
baseline_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]

# different levels of padding applied - padding_df
padding_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['partial_gauss'] == 0)]

# keep amd on and off - amd_df
amd_df = df[(df['rcm'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]

# keep rcm on and off - rcm_df
rcm_df = df[(df['amd'] == False) & (df['partial_gauss'] == 0) & (df['padding'] == 0)]

# combined approaches
pg_df = df[(df['amd'] == False) & (df['rcm'] == False) & (df['padding'] == 0)]

rcm_padding_df = df[(df['amd'] == False) & (df['partial_gauss'] == 0)]

pg_padding_df = df[(df['amd'] == False) & (df['rcm'] == False)]

amd_pg_df = df[(df['rcm'] == False) & (df['padding'] == 0)]

pg_rcm_df = df[(df['amd'] == False) & (df['padding'] == 0)]

# three-way approaches
amd_pg_padding_df = df[df['rcm'] == False]

amd_pg_rcm_df = df[df['padding'] == 0]

pg_padding_rcm_df = df[df['amd'] == False]

# 4-way
all_combined_df = df

df_list = [baseline_df, padding_df, amd_df, rcm_df, pg_df, rcm_padding_df, pg_padding_df, amd_pg_df, pg_rcm_df, amd_pg_padding_df, amd_pg_rcm_df, pg_padding_rcm_df, all_combined_df]
df_str_list = ["baseline_df", "padding_df", "amd_df", "rcm_df", "pg_df", "rcm_padding_df", "pg_padding_df", "amd_pg_df", "pg_rcm_df", "amd_pg_padding_df", "amd_pg_rcm_df", "pg_padding_rcm_df", "all_combined_df"]


In [38]:
df.tail()

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n,log_obj_func,obj_func,reduced_variable_percentage
9403,True,True,2,Pres_Poisson,623417.0,5.0,1625.0,73.0,718845.0,73.0,NaN,14819.0,25.835484,1.660383e+11,0.000337
9404,True,True,2,Pres_Poisson,623417.0,5.0,325.0,203.0,718845.0,203.0,NaN,14819.0,31.879337,6.998744e+13,0.000337
9405,True,True,2,Pres_Poisson,623417.0,5.0,85.0,511.0,718845.0,511.0,NaN,14819.0,37.418218,1.780433e+16,0.000337
9406,True,True,2,Pres_Poisson,623417.0,5.0,19.0,2117.0,718845.0,2117.0,NaN,14819.0,45.946532,9.001713e+19,0.000337
9407,True,True,2,Pres_Poisson,623417.0,5.0,1.0,14819.0,718845.0,14819.0,NaN,14819.0,57.621993,1.059042e+25,0.000337


In [39]:
# setup color map for each matrix
matrix_names = df["matrix_name"].unique().tolist()
num_matrices = len(matrix_names)

colorscale = px.colors.qualitative.Plotly
colors = [colorscale[i % len(colorscale)] for i in range(num_matrices)]

matrix_color_map = {matrix:color for matrix, color in zip(matrix_names, colors)}

In [40]:
# normality tests - tests the null hypothesis that a sample comes from a normal distribution
# small p-value == this may be taken as evidence against the null hypothesis in favor of the alternative: the weights were not drawn from a normal distribution. 
# Note that:
# The inverse is not true; that is, the test is not used to provide evidence for the null hypothesis.

for df, df_str in zip(df_list, df_str_list):
    res = normaltest(df["obj_func"])
    print("{} normality, p-value: {}".format(df_str, res.pvalue))


baseline_df normality, p-value: 3.7723043882067505e-28
padding_df normality, p-value: 6.902223603472822e-180
amd_df normality, p-value: 1.2930279380469287e-43
rcm_df normality, p-value: 1.2930277107836354e-43
pg_df normality, p-value: 0.0
rcm_padding_df normality, p-value: 0.0
pg_padding_df normality, p-value: 0.0
amd_pg_df normality, p-value: 0.0
pg_rcm_df normality, p-value: 0.0
amd_pg_padding_df normality, p-value: 0.0
amd_pg_rcm_df normality, p-value: 0.0
pg_padding_rcm_df normality, p-value: 0.0
all_combined_df normality, p-value: 0.0


In [41]:
for df, df_str in zip(df_list, df_str_list):
    res = normaltest(df["log_obj_func"])
    print("{} normality, p-value: {}".format(df_str, res.pvalue))

baseline_df normality, p-value: 0.16735531412714397
padding_df normality, p-value: 2.224460847772188e-10
amd_df normality, p-value: 0.033787221207441334
rcm_df normality, p-value: 0.0350559964103515
pg_df normality, p-value: 0.0
rcm_padding_df normality, p-value: 3.581951875040232e-20
pg_padding_df normality, p-value: 0.0
amd_pg_df normality, p-value: 0.0
pg_rcm_df normality, p-value: 0.0
amd_pg_padding_df normality, p-value: 0.0
amd_pg_rcm_df normality, p-value: 0.0
pg_padding_rcm_df normality, p-value: 0.0
all_combined_df normality, p-value: 0.0


Conclusion: the (log) runtime estimates do not follow a normal distribution. Therefore, I need to work with non-parametric tests when comparing the approaches.

In [42]:
# Next: check if different approaches give better results than baseline
# important: we only care about best tile size choice rows that get enabled by these approaches

# null-hyp: no difference
_, p_value = mannwhitneyu(rcm_df[rcm_df["rcm"] == False]["log_obj_func"], rcm_df[rcm_df["rcm"] == True]["log_obj_func"], alternative='two-sided')
print(p_value)

0.9706139971016378


In [43]:
rcm_df[rcm_df["rcm"] == False]["log_obj_func"].describe()

count    57.000000
mean     34.356460
std       9.327507
min      20.255555
25%      28.135807
50%      32.908784
75%      40.513622
max      57.623207
Name: log_obj_func, dtype: float64

In [44]:
rcm_df[rcm_df["rcm"] == True]["log_obj_func"].describe()

count    57.000000
mean     34.330409
std       9.371346
min      20.019984
25%      28.148473
50%      32.908784
75%      40.513622
max      57.623207
Name: log_obj_func, dtype: float64

In [45]:
# get best tile size choices
idx = rcm_padding_df.groupby(by=['matrix_name', 'rcm'])['log_obj_func'].idxmin()
rcm_padding_tile_agg_df = rcm_padding_df.loc[idx].reset_index(drop=True)
rcm_padding_tile_agg_df

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n,log_obj_func,obj_func,reduced_variable_percentage
0,False,False,1,Pres_Poisson,715804.0,0.0,2599.0,61.0,715804.0,61.0,NaN,14823.0,25.070247,7.724491e+10,0.0
1,False,True,1,Pres_Poisson,715804.0,0.0,2105.0,61.0,715804.0,61.0,NaN,14823.0,24.949895,6.848601e+10,0.0
2,False,False,6,bcsstk13,70264.0,0.0,569.0,41.0,70264.0,41.0,0.982487,2009.0,22.397285,5.333563e+09,0.0
3,False,True,6,bcsstk13,70264.0,0.0,665.0,41.0,70264.0,41.0,0.982487,2009.0,22.435137,5.539317e+09,0.0
4,False,False,10,ex10,43497.0,0.0,1031.0,20.0,43497.0,20.0,0.992511,2420.0,20.024970,4.974324e+08,0.0
5,False,True,5,ex10,43497.0,0.0,727.0,23.0,43497.0,23.0,0.992511,2415.0,19.894237,4.364731e+08,0.0
6,False,False,2,ex10hs,42944.0,0.0,1400.0,17.0,42944.0,17.0,0.993385,2550.0,20.208191,5.974558e+08,0.0
7,False,True,2,ex10hs,42944.0,0.0,1218.0,17.0,42944.0,17.0,0.993385,2550.0,19.944256,4.588600e+08,0.0
8,False,False,6,ex13,75392.0,0.0,1209.0,18.0,75392.0,18.0,0.988568,2574.0,20.058993,5.146478e+08,0.0
9,False,True,6,ex13,75392.0,0.0,1291.0,18.0,75392.0,18.0,0.988568,2574.0,20.181201,5.815460e+08,0.0


Explore effects of rcm followed by padding

do hypothesis testing: 
- null hypothesis: applying RCM on top of padding does not influence the runtime estimate
- significance level: 0.01
- need to check assumptions: observations independent, from sample to sample, samples were obtained via random sampling, data should be approximately normally distributed, variances should be approximately equal (if not, just use Welch’s t-test).

In [46]:
# considering only the best tile size choices
rcm_on_group = rcm_padding_tile_agg_df[rcm_padding_tile_agg_df['rcm'] == True]
rcm_off_group = rcm_padding_tile_agg_df[rcm_padding_tile_agg_df['rcm'] == False]

print("mean, std for log_obj_func without rcm: ", rcm_on_group["log_obj_func"].mean(), rcm_on_group["log_obj_func"].std())
print("mean, std for log_obj_func with rcm: ", rcm_off_group["log_obj_func"].mean(), rcm_off_group["log_obj_func"].std())

mean, std for log_obj_func without rcm:  21.11574241409493 1.947051813991641
mean, std for log_obj_func with rcm:  21.28768474337918 1.9424158567058876


In [47]:
# considering all tile size choices
rcm_on_group = rcm_padding_df[rcm_padding_df['rcm'] == True]
rcm_off_group = rcm_padding_df[rcm_padding_df['rcm'] == False]

print("mean, std for log_obj_func without rcm: ", rcm_on_group["log_obj_func"].mean(), rcm_on_group["log_obj_func"].std())
print("mean, std for log_obj_func with rcm: ", rcm_off_group["log_obj_func"].mean(), rcm_off_group["log_obj_func"].std())

mean, std for log_obj_func without rcm:  34.15324668571295 9.347270650081919
mean, std for log_obj_func with rcm:  34.17355178471767 9.331924756941827


In [48]:
rcm_on_group

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n,log_obj_func,obj_func,reduced_variable_percentage
40308,False,True,8,ex13,75392.0,0.0,33626.0,2.0,75392.0,23.0,0.988568,2576.0,27.118029,5.987016e+11,0.0
40309,False,True,8,ex13,75392.0,0.0,11215.0,4.0,75392.0,23.0,0.988568,2576.0,24.925270,6.682011e+10,0.0
40310,False,True,8,ex13,75392.0,0.0,4521.0,7.0,75392.0,23.0,0.988568,2576.0,23.122570,1.101551e+10,0.0
40311,False,True,8,ex13,75392.0,0.0,3971.0,8.0,75392.0,23.0,0.988568,2576.0,22.867801,8.538068e+09,0.0
40312,False,True,8,ex13,75392.0,0.0,1816.0,14.0,75392.0,23.0,0.988568,2576.0,21.372825,1.914697e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5368,False,True,3,Pres_Poisson,715804.0,0.0,66067.0,5.0,715804.0,593.0,NaN,14825.0,38.346162,4.503254e+16,0.0
5369,False,True,3,Pres_Poisson,715804.0,0.0,8627.0,25.0,715804.0,593.0,NaN,14825.0,38.311809,4.351184e+16,0.0
5370,False,True,3,Pres_Poisson,715804.0,0.0,73.0,593.0,715804.0,593.0,NaN,14825.0,38.311167,4.348388e+16,0.0
5371,False,True,3,Pres_Poisson,715804.0,0.0,13.0,2965.0,715804.0,2965.0,NaN,14825.0,47.967794,6.794354e+20,0.0


In [49]:
rcm_padding_diff_df = pd.merge(rcm_on_group, rcm_off_group, on=['amd', 'padding', 'matrix_name', 'partial_gauss', 'tile_size'], suffixes=('_on', '_off'))

# "on" is the improved version, we want diff to be positive if runtime is increased, while negative if runtime decreased
# if "on" is lower than "off" then we made an improvement since the runtime has decreased
# thus we need to have "on" minus "off" so that "on" - "off" < 0 when "on" < "off"
rcm_padding_diff_df["log_obj_func_diff"] = rcm_padding_diff_df["log_obj_func_on"] - rcm_padding_diff_df["log_obj_func_off"]
rcm_padding_diff_df["log_obj_func_diff"].describe()

count    6.150000e+02
mean    -2.030510e-02
std      1.488973e-01
min     -9.831617e-01
25%     -1.641758e-03
50%      0.000000e+00
75%      1.523345e-08
max      8.163619e-01
Name: log_obj_func_diff, dtype: float64

In [50]:
rcm_padding_df.groupby(by=['matrix_name', 'rcm'])["log_obj_func"].mean()

matrix_name   rcm  
Pres_Poisson  False    40.383752
              True     40.319154
bcsstk13      False    34.170528
              True     34.239608
ex10          False    31.322708
              True     31.290497
ex10hs        False    31.069110
              True     30.990379
ex13          False    32.887519
              True     32.936490
ex15          False    37.166383
              True     37.108467
ex3           False    31.327238
              True     31.338405
Name: log_obj_func, dtype: float64

In [51]:
rcm_padding_df.groupby(by=['matrix_name', 'rcm'])["log_obj_func"].std()

matrix_name   rcm  
Pres_Poisson  False    9.905740
              True     9.956503
bcsstk13      False    7.204764
              True     7.136327
ex10          False    8.289907
              True     8.319182
ex10hs        False    8.866506
              True     8.931551
ex13          False    8.657908
              True     8.616060
ex15          False    9.048029
              True     9.117383
ex3           False    8.310256
              True     8.309291
Name: log_obj_func, dtype: float64

In [52]:
fig = px.histogram(rcm_padding_diff_df, x="log_obj_func_diff", nbins=100, log_y=True, 
                   color="matrix_name",
                #  histnorm="probability",  # won't add up to one overall when multiple groups are used
                  color_discrete_map=matrix_color_map,
                  labels={
                      "log_obj_func_diff": r'$\Delta \log(I^6 + rI^3 + r^2I^2)$',
                      "matrix_name": "Matrix name",
                      "tile_size": "Tile size",
                  })
fig.update_layout(
        title={
            'text': "Change in runtime estimates after adding RCM to Padding",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
fig.write_image("plots/rcm_padding_log_obj_func_diff_plot.pdf")

In [53]:
# this initial analysis indicates that RCM on top of padding generally helps a bit, sometimes it makes things worse. Most often, it does not lead to significant changes. For more understanding, would need to look at separate cases in more detail, skip for now.
# Now I just have to make this visually nice and write it down in the report.

In [54]:
# todo: perhaps I can do the above difference plot as follows:
# - consider one improvement (e.g. rcm)
# - against a set of different baselines - use color for these, see which of the baselines improve the most with added stuff
# - this also allows us to evaluate combined baselines easier

PG then Padding

In [55]:
# here we have enough data samples, only consider best tile size choices
# for now, consider PG as baseline - see how adding Padding effects the situation
pg_padding_df

,amd,rcm,padding,matrix_name,z_full,partial_gauss,rank,tile_size,z_reduced,max_mode_size,sparsity_ratio,n,log_obj_func,obj_func,reduced_variable_percentage
0,False,False,10,bcsstk13,70264.0,0.0,18672.0,3.0,70264.0,61.0,0.982487,2013.0,27.933391,1.353061e+12,0.000000
1,False,False,10,bcsstk13,70264.0,0.0,3355.0,11.0,70264.0,61.0,0.982487,2013.0,25.268320,9.416556e+10,0.000000
2,False,False,10,bcsstk13,70264.0,0.0,755.0,33.0,70264.0,61.0,0.982487,2013.0,24.708777,5.381281e+10,0.000000
3,False,False,10,bcsstk13,70264.0,0.0,339.0,61.0,70264.0,61.0,0.982487,2013.0,24.674989,5.202494e+10,0.000000
4,False,False,10,bcsstk13,70264.0,0.0,75.0,183.0,70264.0,183.0,0.982487,2013.0,31.256934,3.755900e+13,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,False,False,3,Pres_Poisson,623389.0,5.0,23.0,2964.0,717825.0,2964.0,NaN,14820.0,47.965770,6.780617e+20,0.000337
5478,False,False,3,Pres_Poisson,623389.0,5.0,16.0,3705.0,717825.0,3705.0,NaN,14820.0,49.304631,2.586600e+21,0.000337
5479,False,False,3,Pres_Poisson,623389.0,5.0,9.0,4940.0,717825.0,4940.0,NaN,14820.0,51.030724,1.453321e+22,0.000337
5480,False,False,3,Pres_Poisson,623389.0,5.0,4.0,7410.0,717825.0,7410.0,NaN,14820.0,53.463514,1.655424e+23,0.000337


In [56]:
idx = pg_padding_df.groupby(by=['matrix_name', 'padding', 'partial_gauss'])['log_obj_func'].idxmin()
pg_padding_tile_agg_df = pg_padding_df.loc[idx].reset_index(drop=True)

pg_padding_tile_agg_df["log_obj_func"] = round(pg_padding_tile_agg_df["log_obj_func"], 2)

In [57]:
matrix_name = "ex3"
pg_from = 990
fig = px.density_heatmap(pg_padding_tile_agg_df[(pg_padding_tile_agg_df["matrix_name"] == matrix_name) & (pg_padding_tile_agg_df["partial_gauss"] >= pg_from) & (pg_padding_tile_agg_df["partial_gauss"] <= pg_from+10)], x="partial_gauss", y="padding", z='log_obj_func',  histfunc="min",
                         nbinsx=11, nbinsy=11, text_auto=True,
                         # symbol="log_obj_func",
                 # color_discrete_sequence=colors, 
                 # trendline="ols", trendline_scope="overall", trendline_color_override="lime", trendline_options=dict(log_y=True),
                 # symbol="matrix_name", color_discrete_map=matrix_color_map,
              labels={
                     "padding": "Padding",
                     "partial_gauss": "Number of variables eliminated by PG",
                  "log_obj_func": "log(Eq. 3.9)",
                  "num_possible_tiles": "Number of possible<br>tile size choices"
                 }
              )
fig.update_layout(
    title={
        'text': "Influence of variable elimination with padding on TT-MALS runtime for {}".format(matrix_name),
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='white',   # Plot area background color
    paper_bgcolor='white',  # Entire figure background color
    font=dict(color='black'), # Font color
    # yaxis_title=r'$\log(I^6 + rI^3 + r^2I^2)$'
)
fig.show()
fig.write_image("plots/{}_pg_from{}_padding_heatmap_log_obj_func.pdf".format(matrix_name, pg_from))

AMD then PG

In [70]:
# exclude ex15, and Pres_Poisson as those have much fewer samples for this than the others
amd_pg_df = amd_pg_df[(amd_pg_df["matrix_name"] != "ex15") & (amd_pg_df["matrix_name"] != "Pres_Poisson")]

In [71]:
amd_pg_df_diff_df = pd.merge(amd_pg_df[amd_pg_df["amd"] == True], amd_pg_df[amd_pg_df["amd"] == False], on=['rcm', 'padding', 'matrix_name', 'partial_gauss', 'tile_size'], suffixes=('_on', '_off'))
amd_pg_df_diff_df["log_obj_func_diff"] = amd_pg_df_diff_df["log_obj_func_on"] - amd_pg_df_diff_df["log_obj_func_off"]

In [76]:
fig = px.histogram(amd_pg_df_diff_df, x="log_obj_func_diff", nbins=100, log_y=True, 
                   color="matrix_name",
                #  histnorm="probability",  # won't add up to one overall when multiple groups are used
                  color_discrete_map=matrix_color_map,
                  labels={
                      "log_obj_func_diff": r'$\Delta \log(I^6 + rI^3 + r^2I^2)$',
                      "matrix_name": "Matrix name",
                      "tile_size": "Tile size",
                  })
fig.update_layout(
        title={
            'text': "Change in runtime estimates when adding AMD before PG",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
fig.write_image("plots/amd_pg_log_obj_func_diff_all_tiles_plot.pdf")

In [77]:
# now consider only the best tile size choice scenarios
idx = amd_pg_df.groupby(by=['matrix_name', 'amd', 'partial_gauss'])['log_obj_func'].idxmin()
amd_pg_df_tile_agg_df = amd_pg_df.loc[idx].reset_index(drop=True)

In [78]:
amd_pg_df_tile_agg_diff_df = pd.merge(amd_pg_df_tile_agg_df[amd_pg_df_tile_agg_df["amd"] == True], amd_pg_df_tile_agg_df[amd_pg_df_tile_agg_df["amd"] == False], on=['rcm', 'padding', 'matrix_name', 'partial_gauss', 'tile_size'], suffixes=('_on', '_off'))
amd_pg_df_tile_agg_diff_df["log_obj_func_diff"] = amd_pg_df_tile_agg_diff_df["log_obj_func_on"] - amd_pg_df_tile_agg_diff_df["log_obj_func_off"]

In [79]:
fig = px.histogram(amd_pg_df_tile_agg_diff_df, x="log_obj_func_diff", nbins=100, log_y=True, 
                   color="matrix_name",
                #  histnorm="probability",  # won't add up to one overall when multiple groups are used
                  color_discrete_map=matrix_color_map,
                  labels={
                      "log_obj_func_diff": r'$\Delta \log(I^6 + rI^3 + r^2I^2)$',
                      "matrix_name": "Matrix name",
                      "tile_size": "Tile size",
                  })
fig.update_layout(
        title={
            'text': "Change in runtime estimates when adding AMD before PG",
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        plot_bgcolor='white',  # Plot area background color
        paper_bgcolor='white',  # Entire figure background color
        font=dict(color='black'),  # Font color
    )
fig.show()
fig.write_image("plots/amd_pg_log_obj_func_diff_best_tile_only_plot.pdf")